In [1]:
#standard imports and setups used during EAE127
import math
import numpy as np 
import os
import matplotlib.pyplot as plt 
import matplotlib.lines as mlines
#Disable Python Warning Output
#(NOTE: Only for production, comment out for debugging)
import warnings
warnings.filterwarnings('ignore')
### PLOTTING DEFAULTS BOILERPLATE (OPTIONAL) #########################
#SET DEFAULT FIGURE APPERANCE
import seaborn as sns #Fancy plotting package 
#No Background fill, legend font scale, frame on legend
sns.set_theme(style='whitegrid', font_scale=1.5, rc={'legend.frameon': True})
#Mark ticks with border on all four sides (overrides 'whitegrid')
sns.set_style('ticks')
#ticks point in
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
#fix invisible marker bug
sns.set_context(rc={'lines.markeredgewidth': 0.1})
#restore default matplotlib colormap
mplcolors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
'#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
sns.set_palette(mplcolors)

#Get color cycle for manual colors
colors = sns.color_palette()
#SET MATPLOTLIB DEFAULTS
#(call after seaborn, which changes some defaults)
params = {
#FONT SIZES
'axes.labelsize' : 30, #Axis Labels
'axes.titlesize' : 30, #Title
'font.size' : 28, #Textbox
'xtick.labelsize': 22, #Axis tick labels
'ytick.labelsize': 22, #Axis tick labels
'legend.fontsize': 15, #Legend font size
'font.family' : 'serif',
'font.fantasy' : 'xkcd',
'font.sans-serif': 'Helvetica',
'font.monospace' : 'Courier',
#AXIS PROPERTIES
'axes.titlepad' : 2*6.0, #title spacing from axis
'axes.grid' : True, #grid on plot
'figure.figsize' : (8,8), #square plots
'savefig.bbox' : 'tight', #reduce whitespace in saved figures
#LEGEND PROPERTIES
'legend.framealpha' : 0.5,
'legend.fancybox' : True,
'legend.frameon' : True,
'legend.numpoints' : 1,
'legend.scatterpoints' : 1,
'legend.borderpad' : 0.1,
'legend.borderaxespad' : 0.1,
'legend.handletextpad' : 0.2,
'legend.handlelength' : 1.0,
'legend.labelspacing' : 0,
}
import matplotlib #type:ignore
matplotlib.rcParams.update(params) #update matplotlib defaults, call after￿
### END OF BOILERPLATE ##################################################
colors = sns.color_palette() #color cycle

In [ ]:
#constants

#efficiency
etaC = 0.88
etaB = 0.95
etaT = 0.86
etaF = 0.91
etaM = 0.993
etaN = 0.94
etaFN = 0.94
etaAb = 0.9

#total pressure ratio
piC = 17
piF = 2.015127
piD = 0.93
piB = 0.91
piM = 0.98
piAb = 0.96

#ideal heat ratio
kA = 1.408327
kD = 1.40424
kC = 1.386146
kF = 1.400929
kT = 1.337778
kN = 1.280625
kFN = 1.397652

#specific heats (BTU/lbm R)
CpF = 0.2395595
CpC = 0.2461057
CpAb = 0.2810447
CpT = 0.2715298
CpN = 0.3128676
CpFN = 0.2409686
CpB = 0.2677774
CpMC = 0.2569374
CpMu = 0.2466467

deltaH = 18100 #BTU/lbm
Tt4 = 2230 #R
mdot = 157 #lbm/s
Tt6 = 3300 #R
sigma = 0.5
alpha = 1.6
Ma = 0.93

Ta = 420.77 #R
Pa = 4.893503 #psia



In [ ]:
#main code

def calcThrust(etaC=etaC,etaF=etaF,etaT=etaT,etaB=etaB,piB=piB,piD=piD,etaN=etaN):

    #